<a href="https://colab.research.google.com/github/tntnu/LoRA_fine-tuning/blob/main/%5BNLP%5D_%ED%8C%90%EB%8B%A4%EC%8A%A4%EC%8A%A4%ED%8A%9C%EB%94%94%EC%98%A4_LoRA_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 공개 데이터셋 다운로드

In [4]:
!pip install -U datasets==2.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.21.0 requires datasets>=3.0.0, but you have datasets 2.17.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.10.0 which is incompatible.


In [1]:
from datasets import load_dataset

dataset = load_dataset("royboy0416/ko-alpaca")

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 49620
    })
})


In [2]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}

### microsoft/Phi-4-mini-instruct 데이터셋 포맷

```
<|system|>Insert System Message<|end|><|user|>Insert User Message<|end|><|assistant|>

```



In [3]:
# 'prompt' 필드 생성 함수
def format_instruction(example) :

  system_msg = 'Your are a helpful assistant.'

  # 추가 컨텍스트(input필드)가 있는 경우
  if example['input'] and len(example['input']) > 0 :
    user_msg = f'''{example['instruction']}\n{example['input']}'''

  # input 필드가 없는 경우
  else :
    user_msg = {example['instruction']}

  # 어시스턴트 메시지(output)
  assistant_msg = example['output']

  # Phi-4 포맷 구성
  text = f"<|system|>{system_msg}<|end|><|user|>{user_msg}<|end|><|assistant|>{assistant_msg}"

  return {'prompt' : text}

# 데이터셋의 prompt 필드 업데이트
dataset = dataset.map(format_instruction)

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [4]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'prompt': "<|system|>Your are a helpful assistant.<|end|><|user|>{'건강을 유지하기 위한 세 가지 팁을 알려주세요.'}<|end|><|assistant|>세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다."}

In [5]:
dataset['train'][5]

{'instruction': '홀수 중 하나를 밝히세요.',
 'input': '트위터, 인스타그램, 텔레그램',
 'output': '텔레그램입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n홀수 중 하나를 밝히세요.\n\n### Input:\n트위터, 인스타그램, 텔레그램\n\n### Response:\n텔레그램입니다.',
 'prompt': '<|system|>Your are a helpful assistant.<|end|><|user|>홀수 중 하나를 밝히세요.\n트위터, 인스타그램, 텔레그램<|end|><|assistant|>텔레그램입니다.'}

### 모델 로드 및 튜닝:
1. 모델 학습: gemma-2b 모델을 로드하고, 준비된 데이터셋을 사용하여 모델을 세부 튜닝합니다. 이 과정에서는 학습률, 에폭 수 등의 파라미터를 조정할 수 있습니다.
2. 평가 및 반복: 튜닝된 모델을 평가하고 결과를 확인합니다. 필요에 따라 여러 번 반복하여 모델의 성능을 최적화할 수 있습니다.

In [1]:
!pip install accelerate peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.1 MB/s eta 0:00:00


In [3]:
import accelerate, peft, trl
print("accelerate:", accelerate.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)

import bitsandbytes as bnb
print(bnb.__version__)

accelerate: 1.9.0
peft: 0.17.0
trl: 0.21.0
0.47.0


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, BitsAndBytesConfig, TrainingArguments, pipeline, logging

from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [7]:
model_id = 'microsoft/Phi-4-mini-instruct'

bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                bnb_4bit_quant_type = 'nf4',
                                bnb_4bit_compute_dtype = torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map = 'auto')

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|endoftext|>'

In [9]:
dataset = dataset.map(lambda samples : tokenizer(samples['prompt']), batched = True)
dataset

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 49620
    })
})

In [10]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'prompt': "<|system|>Your are a helpful assistant.<|end|><|user|>{'건강을 유지하기 위한 세 가지 팁을 알려주세요.'}<|end|><|assistant|>세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.",
 'input_ids': [200022,
  9719,
  553,
  261,
  10297,
  29186,
  13,
  200020,
  200021,
  40200,
  23459,
  28318,
  3281,
  125256,
  33181,
  52294,
  28126,
  65296,
  19839,
  223,
  3281,
  104661,
  94021,
  8856,
  92,
  200020,
  200019,
  11734,
  65296,
  19839,
  223,
  4740,
  12652,
  57622,
  17087,
  67693,
  149600,
  37400,
  247,
  4283,
  16205,
  11,
  176267,
  3748,
  8963,
  9018,
  3281,
  66765,
  17231,
  11,
  176490,
  34555,
  156

In [11]:
dataset = dataset['train'].train_test_split(test_size = 0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 39696
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 9924
    })
})

In [12]:
train_data = dataset['train']
test_data = dataset['test']

In [13]:
print(train_data[0])

{'instruction': '"무결성"의 정의를 가장 잘 설명하는 단어는 무엇인가요?', 'input': '', 'output': "'무결성'은 영어로 integrity로 처리될 수 있으며, 가장 최소한의 요소로의 무작위 수정 없이 데이터나 정보의 완전성과 일관성을 보유하는 상태를 의미합니다.", 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n"무결성"의 정의를 가장 잘 설명하는 단어는 무엇인가요?\n\n### Response:\n\'무결성\'은 영어로 integrity로 처리될 수 있으며, 가장 최소한의 요소로의 무작위 수정 없이 데이터나 정보의 완전성과 일관성을 보유하는 상태를 의미합니다.', 'prompt': '<|system|>Your are a helpful assistant.<|end|><|user|>{\'"무결성"의 정의를 가장 잘 설명하는 단어는 무엇인가요?\'}<|end|><|assistant|>\'무결성\'은 영어로 integrity로 처리될 수 있으며, 가장 최소한의 요소로의 무작위 수정 없이 데이터나 정보의 완전성과 일관성을 보유하는 상태를 의미합니다.', 'input_ids': [200022, 9719, 553, 261, 10297, 29186, 13, 200020, 200021, 90, 27386, 14812, 27188, 9360, 1, 3408, 153254, 4831, 54530, 47305, 89157, 14108, 38340, 5959, 2770, 103740, 149298, 7952, 30, 17352, 200020, 200019, 6, 14812, 27188, 9360, 6, 4740, 178051, 3710, 27866, 3710, 98436, 57189, 8963, 96864, 11, 54530, 162462, 3748, 

In [33]:
def get_completion(query : str, model, tokenizer) :

  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>''' # 모델에 따라 바뀜

  prompt = prompt_template.format(query = query)
  encodeds = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encodeds.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# Fine-tuning 이전
result = get_completion(query ="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)

Your are a helpful assistant.건강을 유지하기 위한 세 가지 팁을 알려주세요.1. 균일한 식사: 균일한 식사, 영양이 풍부한 식품을 섭취하고 과식하지 않도록 하세요.
2. 운동: 일상 생활에서 일찍 일어나서 일찍 일어나서 일찍 일어나도록 하세요. 최소 30분의 운동을 일주일에 3회 이상 수행하세요.
3. 휴식: 하루에 최소 8시간의 수면을 취하세요. 스트레스를 줄이기 위해 휴식을 취하세요.


### 모델 구조 확인(target_modules)

In [17]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.mlp
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.activation_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.0.resid_attn_dropout
model.layers.0.resid_mlp_dropout
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.mlp
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.activation_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.1.resid_attn_dropout
model.layers.1.resid_mlp_dropout
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.mlp
model.layers.2.mlp.gate_up_proj
model.layers.2.mlp.down_proj
model.layers.2.mlp.activation_fn
model.layers.2.input_layerno

In [ ]:
import trl as SFTTrainer
print(bnb.__version__)

In [25]:
def copy_prompt_to_text(example):
    example['text'] = example['prompt']
    return example

train_data = train_data.map(copy_prompt_to_text)
test_data = test_data.map(copy_prompt_to_text)

Map:   0%|          | 0/39696 [00:00<?, ? examples/s]

Map:   0%|          | 0/9924 [00:00<?, ? examples/s]

In [32]:
train_data

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 39696
})

In [27]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [34]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r = 32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout = 0.05,
    task_type = 'CAUSAL_LM'
)

model = model.unload()
model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

trainer = SFTTrainer(
    model = model,
    train_dataset = train_data,
    eval_dataset = test_data,
    peft_config = lora_config,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 50,
        learning_rate = 3e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = 'outputs',
        optim = 'paged_adamw_8bit'
    ),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)

)
trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
1,3.911800
2,4.189800
3,3.163200
4,2.869700
5,2.999900
6,3.085800
7,3.331000
8,3.168000
9,2.848400
10,3.276900


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=50, training_loss=2.463671138286591, metrics={'train_runtime': 95.7482, 'train_samples_per_second': 2.089, 'train_steps_per_second': 0.522, 'total_flos': 309724218261504.0, 'train_loss': 2.463671138286591})

In [35]:
# fine-tuning 이후
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

Your are a helpful assistant.건강을 유지하기 위한 세 가지 팁을 알려주세요.1. 균일한 식사, 적절한 운동, 충분한 수면을 유지하세요.
2. 스트레스를 줄이기 위해 명상, 요가 또는 다른 명상 기술을 시도하세요.
3. 건강한 생활 습관을 유지하기 위해 건강한 식사, 적절한 운동, 충분한 수면을 유지하세요.


In [36]:
# Fine tuning 이후
result = get_completion(query="불면증을 해결하는 방법을 세 가지 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

Your are a helpful assistant.불면증을 해결하는 방법을 세 가지 알려주세요.1. 충분한 수면을 취하는 것
2. 수면 환경을 개선하는 것
3. 수면 습도가 높아지도록 주의하는 것


In [37]:
# 모델 저장

new_model = "microsoft/Phi-4-mini-instruct-it-koalpaca-finetuned"
trainer.model.save_pretrained(new_model)